In [1]:
import tensorflow as tf
d_l=tf.keras.layers.Dense(50,activation='relu',kernel_initializer='he_normal')
#he_normal helps solve vanishing gradient for activation functions like ReLU

leaky_relu=tf.keras.layers.LeakyReLU(alpha=0.2)
dense=tf.keras.layers.Dense(50,activation=leaky_relu,kernel_initializer='he_normal')

fashion_mnist=tf.keras.datasets.fashion_mnist.load_data()
(x_train_full,y_train_full),(x_test_full,y_test_full)=fashion_mnist

x_train,y_train=x_train_full[:-5000],y_train_full[:-5000]
x_valid,y_valid=x_train_full[-5000:],y_train_full[-5000:]

#BATCH NORMALIZATION
#here we add BN layer after every hidden layer activation function
model=tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28,28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300,activation='relu',kernel_initializer='he_normal'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100,activation='relu',kernel_initializer='he_normal'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10,activation='softmax')
    
])
print('Model summary:')
print(model.summary())
#first BN layer has 3136 as 4x784(each BN layer has 4 parameters)

#GRADIENT CLIPPING clips gradient to prevent exploding gradients problem
#optimizer=tf.keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss='sparse_categorical_crossentropy',metrics='accuracy',optimizer='SGD')
#optimizer clips gradient between -1.0 and 1.0



Model summary:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_2 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_3 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)

In [2]:
#TRANSFER LEARNING
history=model.fit(x_train,y_train,epochs=10,validation_data=[x_valid,y_valid])
model.save('keras_fashion',save_format='tf')

model_a=tf.keras.models.load_model('keras_fashion')
#we are going to use this model to train a binary classifier
model_b_on_a=tf.keras.Sequential(model_a.layers[:-1])
model_b_on_a.add(tf.keras.layers.Dense(1,activation='sigmoid'))
#instead you could do transfer learning on model_a clone
model_a_clone=tf.keras.models.clone_model(model_a)
model_a_clone.set_weights(model_a.get_weights())

model_b_on_a=tf.keras.Sequential(model_a_clone.layers[:-1])
model_b_on_a.add(tf.keras.layers.Dense(1,activation='sigmoid'))

for layer in model_b_on_a.layers[:-1]:
    layer.trainable=False #freezing layers
#freezing layers as initially, newly added output layer has no knowledge of task so gives wrong predictions
#this can destabilize model because of backpropogation

optimizer=tf.keras.optimizers.SGD(learning_rate=0.001)
model_b_on_a.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
#we have to compile model everytime we freeze or unfreeze


Epoch 1/10
1719/1719 [==============================] - 12s 6ms/step - loss: 0.5618 - accuracy: 0.8055 - val_loss: 0.4135 - val_accuracy: 0.8574
Epoch 2/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4135 - accuracy: 0.8534 - val_loss: 0.3687 - val_accuracy: 0.8662
Epoch 3/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3690 - accuracy: 0.8674 - val_loss: 0.3573 - val_accuracy: 0.8702
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3461 - accuracy: 0.8739 - val_loss: 0.3471 - val_accuracy: 0.8752
Epoch 5/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3258 - accuracy: 0.8812 - val_loss: 0.3317 - val_accuracy: 0.8774
Epoch 6/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3085 - accuracy: 0.8890 - val_loss: 0.3307 - val_accuracy: 0.8786
Epoch 7/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2919 - accuracy: 0.8955 - val_loss: 0.3279 - val_accuracy

In [3]:
#OPTIMIZERS
#Momentum optimizer

optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
#momentum is beta value
optimizer=tf.keras.optimizers.SGD(learning_rate=0.001,momentum=0.9,nesterov=True)

#Adagrad optimizer changes learning rate for every parameter:
optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.001)
#adagrad often slows down learning rate to a very small value

optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001,rho=0.9)
#improvement over Adagrad as it accumulates gradients of only recent iterations(sum of squared gradients for parameter)

#Adam combines RMSprop and momentum
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.99)
#beta_1 tells us how much of previous gradient information is retained in moving average
#beta_2 tells us how much of previous squared gradient information(rmsprop) is retained in moving average

optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9)
#AdaMax focuses only on max of past squared gradients+ normal gradient so no beta_2

optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001,beta_1=0.9,beta_2=0.99)
#Nadam is Adam+nesterov

#LEARNING RATE SCHEDULING
optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, decay=1e-4) #performance scheduling
#here decay=1/s, so after every 1000 instances learning rate changes

def exponential_decay(epoch):
    return 0.1*0.001**(epoch/20) #learnin rate changes after every 20 instances?
#to use above function in callback:

def exponential_decay(lr,s):
    def exponential_decay_fn(epoch):
        return lr*0.001**(epoch/s)
    return(exponential_decay_fn)

print('Evaluation before using learning rate scheduler')
print(model.evaluate(x_test_full,y_test_full))

exponential_decay_fn=exponential_decay(lr=0.001,s=20)
lr_scheduler=tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history=model.fit(x_train,y_train,validation_data=[x_valid,y_valid],epochs=10,callbacks=[lr_scheduler])
print('Evaluation after using learning rate scheduler')
print(model.evaluate(x_test_full,y_test_full))

Evaluation before using learning rate scheduler
313/313 [==============================] - 1s 4ms/step - loss: 0.3402 - accuracy: 0.8815
[0.34021487832069397, 0.8815000057220459]
Epoch 1/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.2361 - accuracy: 0.9155 - val_loss: 0.3138 - val_accuracy: 0.8868 - lr: 0.0010
Epoch 2/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.2338 - accuracy: 0.9158 - val_loss: 0.3069 - val_accuracy: 0.8860 - lr: 7.0795e-04
Epoch 3/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.2305 - accuracy: 0.9162 - val_loss: 0.3090 - val_accuracy: 0.8870 - lr: 5.0119e-04
Epoch 4/10
1719/1719 [==============================] - 11s 7ms/step - loss: 0.2293 - accuracy: 0.9175 - val_loss: 0.3096 - val_accuracy: 0.8868 - lr: 3.5481e-04
Epoch 5/10
1719/1719 [==============================] - 21s 12ms/step - loss: 0.2305 - accuracy: 0.9169 - val_loss: 0.3111 - val_accuracy: 0.8872 - lr: 2.5119e-04
Epoch 6/10
1719

In [4]:
#REGULARIZATION
layer=tf.keras.layers.Dense(100,activation='relu',kernel_initializer='he_normal',
                            kernel_regularizer=tf.keras.regularizers.l2(0.01))

#Dropout
model=tf.keras.Sequential([tf.keras.layers.Flatten(input_shape=[28,28]),
                          tf.keras.layers.Dropout(rate=0.2),
                          tf.keras.layers.Dense(100,activation='relu',kernel_initializer='he_normal'),
                          tf.keras.layers.Dropout(rate=0.2),
                          tf.keras.layers.Dense(100,activation='relu',kernel_initializer='he_normal'),
                          tf.keras.layers.Dropout(rate=0.2),
                          tf.keras.layers.Dense(10,activation='softmax')])
#0.2 means 0.2 probability of each neuron in that layer being dropped or deactivated

#MAX NORM regularization
model_maxnorm=tf.keras.Sequential()
model_maxnorm.add(tf.keras.layers.Flatten(input_shape=[28,28]))
layer=tf.keras.layers.Dense(100,activation='relu',kernel_initializer='he_normal',
                           kernel_constraint=tf.keras.constraints.max_norm(1.0))
#kernel_constraint sets the maximum value of weight
model_maxnorm.add(layer)
model_maxnorm.add(tf.keras.layers.Dense(50,activation='relu',kernel_initializer='he_normal',
                                       kernel_constraint=tf.keras.constraints.max_norm(2.0)))
model_maxnorm.add(tf.keras.layers.Dense(10,activation='softmax'))


In [5]:
#data preprocessing using tensorflow
x=tf.range(10)#creates 1D tensor containing values from 0 to 9
dataset=tf.data.Dataset.from_tensor_slices(x) #creates dataset from the tensor x
print('Created datasdet using tensorflow:')
print(dataset)
#dataset contains 10 zero diensional tensors of type tf.int32, shapes=() as each tensor of dataset is 0 dimensional
#if individual tensor of dataset was a 1 dimensional tensor, then shapes would be length of the 1D tensor

print('Elements of dataset:')
for item in dataset:
    print(item)

x_nested={'a':([1,2,3],[4,5,6]),'b':[7,8,9]}
dataset=tf.data.Dataset.from_tensor_slices(x_nested)
print('Elements of tensorflow dataset created from dictionary')
for item in dataset:
    print(item)
#(corresponding elements of all 3 lists stored as tensors)

#CHAINING TRANSFORMATIONS
dataset=tf.data.Dataset.from_tensor_slices(tf.range(10))
dataset=dataset.repeat(3).batch(7) # repeat means dataset repeated 3 times, now it contains 30 zero dimensional tensors
#batch(7) means each individual tensor in dataset is 1D tensor of max size 7
print('Elements of repeated and batched dataset:')
for item in dataset:
    print(item)

#These dataset methods return new datasets
def square_element(element):
    return element**2


dataset = dataset.map(square_element)
print('Elements of dataset after map() function:')
for item in dataset:
    print(item)
    
#if intensive computation is used via apply, we can distribute it across multiple threads
def calc_sum(x):
    return(x+(x*2))
dataset1=tf.data.Dataset.from_tensor_slices(tf.range(5))
dataset1=dataset1.repeat(3).batch(7)
dataset1=dataset1.map(calc_sum,num_parallel_calls=tf.data.AUTOTUNE)
#this lets tensorflow automatically decide number of threads

print('Elements of dataset1 after using parallel calls concept:')
for item in dataset1:
    print(item)

filtered_dataset=dataset1.filter(lambda x:tf.reduce_sum(x)>13)
print('Batches having sum greater than 13:')
for item in filtered_dataset:
    print(item)

print('Taking only the first 2 batches from your dataset:')
for item in dataset.take(2):
    print(item)

Created datasdet using tensorflow:
<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>
Elements of dataset:
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
Elements of tensorflow dataset created from dictionary
{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=4>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=7>}
{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=5>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=8>}
{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=3>, <tf.Tensor: shape=(), dtype=int32, numpy=6>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=9>}
Ele

In [6]:
#SHUFFLING DATASET
dataset=tf.data.Dataset.range(10).repeat(2)#dataset count of numbers is 20
dataset=dataset.shuffle(buffer_size=4,seed=42).batch(7)
import pandas as pd
print('Shuffled dataset:')
for item in dataset:
    print(item)

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
housing=fetch_california_housing()
x=housing.data
y=housing.target

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train,x_valid,y_train,y_valid=train_test_split(x_train,y_train,test_size=0.2,random_state=42)


train_data=[(x_train,y_train)]
valid_data=[(x_valid,y_valid)]
test_data=[(x_test,y_test)]

train_filepaths=['C:\\Personal\\ML\\csv files\\DL\\train1.csv', 'C:\\Personal\\ML\\csv files\\DL\\train2.csv',
                   'C:\\Personal\\ML\\csv files\\DL\\train3.csv', 'C:\\Personal\\ML\\csv files\\DL\\train4.csv',
                   'C:\\Personal\\ML\\csv files\\DL\\train5.csv', 'C:\\Personal\\ML\\csv files\\DL\\train6.csv',
                   'C:\\Personal\\ML\\csv files\\DL\\train7.csv', 'C:\\Personal\\ML\\csv files\\DL\\train8.csv']

valid_filepaths=['C:\\Personal\\ML\\csv files\\DL\\valid1.csv', 'C:\\Personal\\ML\\csv files\\DL\\valid2.csv',
                   'C:\\Personal\\ML\\csv files\\DL\\valid3.csv', 'C:\\Personal\\ML\\csv files\\DL\\valid4.csv',
                   'C:\\Personal\\ML\\csv files\\DL\\valid5.csv', 'C:\\Personal\\ML\\csv files\\DL\\valid6.csv',
                   'C:\\Personal\\ML\\csv files\\DL\\valid7.csv', 'C:\\Personal\\ML\\csv files\\DL\\valid8.csv']

test_filepaths=['C:\\Personal\\ML\\csv files\\DL\\test1.csv', 'C:\\Personal\\ML\\csv files\\DL\\test2.csv',
                  'C:\\Personal\\ML\\csv files\\DL\\test3.csv', 'C:\\Personal\\ML\\csv files\\DL\\test4.csv',
                  'C:\\Personal\\ML\\csv files\\DL\\test5.csv', 'C:\\Personal\\ML\\csv files\\DL\\test6.csv',
                  'C:\\Personal\\ML\\csv files\\DL\\test7.csv', 'C:\\Personal\\ML\\csv files\\DL\\test8.csv']

num_files=8
num_instances_train=len(x_train)//num_files
print(num_instances_train)
num_instances_valid=len(x_valid)//num_files
num_instances_test=len(x_test)//num_files

for i in range(num_files):
    start_index=i*num_instances_train
    end_index=(i+1)*num_instances_train
    subset_x_train=x_train[start_index:end_index]
    subset_y_train=y_train[start_index:end_index]
    subset_train_data=pd.DataFrame({'MedInc':subset_x_train[:,0],'HouseAge':subset_x_train[:,1],
                                      'AveRooms':subset_x_train[:,2],'AveBedrms':subset_x_train[:,3],
                                      'Population':subset_x_train[:,4],'AveOccup':subset_x_train[:,5],
                                      'Latitude':subset_x_train[:,6],'Longitude':subset_x_train[:,7],
                                      'MedianHouseValue':subset_y_train[:]})
    subset_train_data.to_csv(train_filepaths[i],index=False)

for i in range(num_files): #distributes to 8 csv files for validation set
    start_index=i*num_instances_valid
    end_index=(i+1)*num_instances_valid
    subset_x_valid=x_valid[start_index:end_index]
    subset_y_valid=y_valid[start_index:end_index]
    subset_valid_data=pd.DataFrame({'MedInc':subset_x_valid[:,0],'HouseAge':subset_x_valid[:,1],
                                      'AveRooms':subset_x_valid[:,2],'AveBedrms':subset_x_valid[:,3],
                                      'Population':subset_x_valid[:,4],'AveOccup':subset_x_valid[:,5],
                                      'Latitude':subset_x_valid[:,6],'Longitude':subset_x_valid[:,7],
                                      'MedianHouseValue':subset_y_valid[:]})
    subset_valid_data.to_csv(valid_filepaths[i],index=False)
    
for i in range(num_files): #distributes to 8 csv files for vtest set
    start_index=i*num_instances_test
    end_index=(i+1)*num_instances_test
    subset_x_test=x_test[start_index:end_index]
    subset_y_test=y_test[start_index:end_index]
    subset_test_data=pd.DataFrame({'MedInc':subset_x_test[:,0],'HouseAge':subset_x_test[:,1],
                                      'AveRooms':subset_x_test[:,2],'AveBedrms':subset_x_test[:,3],
                                      'Population':subset_x_test[:,4],'AveOccup':subset_x_test[:,5],
                                      'Latitude':subset_x_test[:,6],'Longitude':subset_x_test[:,7],
                                      'MedianHouseValue':subset_y_test[:]})
    subset_test_data.to_csv(test_filepaths[i],index=False)


Shuffled dataset:
tf.Tensor([1 4 2 3 5 0 6], shape=(7,), dtype=int64)
tf.Tensor([9 8 2 0 3 1 4], shape=(7,), dtype=int64)
tf.Tensor([5 7 9 6 7 8], shape=(6,), dtype=int64)
1651


In [8]:
filepaths_dataset=tf.data.Dataset.list_files(train_filepaths,seed=42)
print(filepaths_dataset)
#.list_files specifically creates dataset from filepaths
n_readers=5
import numpy as np
shuffled_dataset=filepaths_dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
                                             cycle_length=n_readers)
#n_readers means 5 filepaths taken from filepaths_dataset, and each filepath is subjected to lambda function
#TextLineDataset reads lines from each csv file, skip(1) skips column row
#therefore interleave reads one line from every filepath, once filepath exhausted, new ones taken

print('Printing elements of shuffled dataset:')
for line in shuffled_dataset.take(5):
    print(line)
#returns 5 tensors, each containing a byte string

x_mean=np.mean(x_train,axis=0)
x_std=np.std(x_train,axis=0)
print('Mean and standard deviation of every column:')
print(x_mean)

n_inputs=8
#now we convert the byte string into numeric attributes
def parse_csv_file(line):
    defs=[0.]*n_inputs +[tf.constant([],dtype=tf.float32)]
    #defs is a list(1D) with last element being a tensor(used to represent target value) [default values]
    #regular list used to represent input features
    fields=tf.io.decode_csv(line,record_defaults=defs)
    #fields is a list of tensorflow tensors, one for every feature(defs used as deafult value if any missing value detected)
    return tf.stack(fields[:-1]),tf.stack(fields[-1:])
#tf.stack(fields[:-1]) stacks all input tensors into a single tensor

def preprocess(line):
    x,y=parse_csv_file(line)
    x_standardized=(x-tf.constant(x_mean,dtype=tf.float32))/tf.constant(x_std,dtype=tf.float32)
    return x_standardized,y
#above methods help scale the features from the original byte string

first_line=shuffled_dataset.take(1)
print('First line of shuffled dataset after preprocessing:')
print(preprocess(b'4.2083,44.0,5.3232,0.9171,846.0,2.3370,37.47,-122.2,2.782'))


#FUNCTION THAT CONSOLIDATES ALL ABOVE METHODS
def csv_reader_dataset(filepaths,n_readers=5,n_read_threads=None,n_parse_threads=5,shuffle_buffer_size=10_000,
                      seed=42,batch_size=32):
    dataset=tf.data.Dataset.list_files(filepaths,seed=seed)
    dataset=dataset.interleave(
    lambda filepath:tf.data.TextLineDataset(filepath).skip(1),cycle_length=n_readers,num_parallel_calls=n_read_threads)
    dataset=dataset.map(preprocess,num_parallel_calls=n_parse_threads)
    #output of this line is tensor for every row containing numeric version of attributes
    dataset=dataset.shuffle(shuffle_buffer_size,seed=seed)
    return(dataset.batch(batch_size).prefetch(1))
#prefetch makes sure that dataset is always 1 batch ahead
#after batch_size 32 tensors stored in a list


<ShuffleDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>
Printing elements of shuffled dataset:
tf.Tensor(b'2.1836,24.0,3.1074380165289255,1.0611570247933884,2646.0,4.373553719008265,33.98,-118.18,1.62', shape=(), dtype=string)
tf.Tensor(b'3.0707,52.0,4.92326139088729,1.1510791366906474,900.0,2.158273381294964,34.05,-118.38,4.179', shape=(), dtype=string)
tf.Tensor(b'4.7069,27.0,6.523255813953488,1.1162790697674418,873.0,3.383720930232558,38.0,-120.97,1.769', shape=(), dtype=string)
tf.Tensor(b'3.0625,24.0,4.305084745762712,1.0084745762711864,465.0,1.9703389830508475,33.62,-117.89,0.938', shape=(), dtype=string)
tf.Tensor(b'3.4812,52.0,4.520689655172414,1.0689655172413792,707.0,2.4379310344827587,34.06,-118.34,4.328', shape=(), dtype=string)
Mean and standard deviation of every column:
[ 3.86893364e+00  2.85672647e+01  5.42040408e+00  1.09433536e+00
  1.42691650e+03  3.02944025e+00  3.56468476e+01 -1.19583303e+02]
First line of shuffled dataset after preprocessing

In [9]:
train_set=csv_reader_dataset(train_filepaths)
valid_set=csv_reader_dataset(valid_filepaths)
test_set=csv_reader_dataset(test_filepaths)

model=tf.keras.Sequential([tf.keras.layers.Dense(64,activation='relu',input_shape=(8,)),
                          tf.keras.layers.Dense(32,activation='relu'),
                          tf.keras.layers.Dense(1)])
model.compile(loss='mean_squared_error',optimizer='sgd')
model.fit(train_set,validation_data=valid_set,epochs=5)

test_mse=model.evaluate(test_set)
print('Evaluation metrics on test set:')
print(test_mse)
new_set=test_set.take(3)
y_pred=model.predict(new_set)
print('Predictions on new_set:')
print(y_pred[:5])

Epoch 1/5
413/413 [==============================] - 4s 6ms/step - loss: 0.7456 - val_loss: 0.5863
Epoch 2/5
413/413 [==============================] - 2s 5ms/step - loss: 0.4336 - val_loss: 0.4296
Epoch 3/5
413/413 [==============================] - 2s 5ms/step - loss: 0.3905 - val_loss: 0.4303
Epoch 4/5
413/413 [==============================] - 2s 5ms/step - loss: 0.3745 - val_loss: 0.3942
Epoch 5/5
129/129 [==============================] - 1s 3ms/step - loss: 0.3763
Evaluation metrics on test set:
0.3762616813182831
3/3 [==============================] - 0s 23ms/step
Predictions on new_set:
[[1.3673065]
 [2.315952 ]
 [0.8822606]
 [2.8477812]
 [1.5144815]]


In [10]:
#TFRecord
#TFRecord stores data in binary format, the record have varying sizes
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"Example record")
    f.write(b"Example 2 record")

filepaths=["my_data.tfrecord"]
dataset=tf.data.TFRecordDataset(filepaths)
print('Printing elements of TFR dataset:')
for item in dataset:
    print(item)

#KERAS PREPROCESSING
norm_layer=tf.keras.layers.Normalization()
model=tf.keras.Sequential([norm_layer,tf.keras.layers.Dense(1)])
model.compile(loss='mse',optimizer=tf.keras.optimizers.SGD(learning_rate=2e-3))
norm_layer.adapt(x_train)#helps compute mean and standard deviation for normalization
model.fit(x_train,y_train,validation_data=(x_valid,y_valid),epochs=5)
#in this approach preprocessing code is the same for both training and production, done for every epoch

#instead of preprocessing every epoch, we can do it before training only once
norm_layer=tf.keras.layers.Normalization()
norm_layer.adapt(x_train)
x_train_scaled=norm_layer(x_train)
x_valid_scaled=norm_layer(x_valid)
#however this approach does not do preprocessing when in production

final_model=tf.keras.Sequential([norm_layer,model])
x_new=x_test[:3]
y_pred=final_model(x_new)
print('Predictions after creating another sequential model combining preprocessing anf prev model')
print(y_pred)

#we adapt the normalization layer to a regular dataset, we can do this with a tensorflow dataset as well
#norm_layer.adapt(train_set)
train_set=train_set.map(lambda x,y:(norm_layer(x),y))# here you don't need to adapt
#x represents input features, y represents target
print('Trainset after normalization:')
print(train_set)

Printing elements of TFR dataset:
tf.Tensor(b'Example record', shape=(), dtype=string)
tf.Tensor(b'Example 2 record', shape=(), dtype=string)
Epoch 1/5
413/413 [==============================] - 3s 4ms/step - loss: 2.4756 - val_loss: 0.9646
Epoch 2/5
413/413 [==============================] - 1s 3ms/step - loss: 0.7431 - val_loss: 0.6962
Epoch 3/5
413/413 [==============================] - 1s 2ms/step - loss: 0.6358 - val_loss: 0.6499
Epoch 4/5
413/413 [==============================] - 1s 2ms/step - loss: 0.6015 - val_loss: 0.6142
Epoch 5/5
413/413 [==============================] - 1s 2ms/step - loss: 0.5767 - val_loss: 0.5980
Predictions after creating another sequential model combining preprocessing anf prev model
tf.Tensor(
[[-19.512403]
 [-19.23635 ]
 [-19.00037 ]], shape=(3, 1), dtype=float32)
Trainset after normalization:
<MapDataset element_spec=(TensorSpec(shape=(None, 8), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>


In [11]:
#DISCRETIZATION
age=tf.constant([[10.],[93.],[57.],[18.],[37.],[5.]])
print('Tensors age:')
print(age)
discretize_layer=tf.keras.layers.Discretization(bin_boundaries=[18.,50.])
age_categories=discretize_layer(age)
print('Age after discretization:')
print(age_categories)
print('Individual tensors in age_categories:')
for item in age_categories:
    print(item)

#in above snippet we specify the bins
discretize_layer=tf.keras.layers.Discretization(num_bins=3)
discretize_layer.adapt(age)
age_categories=discretize_layer(age)
print('Age categories after specifying only number of bins:')
print(age_categories)

#Category encoding(similar to OneHotEncoder)
onehot_layer=tf.keras.layers.CategoryEncoding(num_tokens=3)#here there are 3 categories
print('After doing CategoryEncoding on age_categories')
print(onehot_layer(age_categories))

onehot_layer=tf.keras.layers.CategoryEncoding(num_tokens=3,output_mode='count')
two_age_categories=np.array([[1,0],[2,2],[2,0]])
print('Conversion of two dimensional age categories to CategoryEncoding:')
print(onehot_layer(two_age_categories))
#here even though a numpy array is given as input, tensorflow converts it to tensor because layers present

#STRINGLOOKUP
cities=['Auckland','Paris','Paris','San Fransisco']
str_lookup=tf.keras.layers.StringLookup()
str_lookup.adapt(cities) # helps identify how many categories there are
str_lookup_res=str_lookup([['Paris'],['Paris'],['Auckland'],['Montreal']])#unknown layers classified as 0

print('Result of stringlookup on new cities data:')
print(str_lookup_res)

#For one hot string lookup
str_lookup_hot=tf.keras.layers.StringLookup(output_mode='one_hot')
str_lookup_hot.adapt(cities)
str_lookup_res=str_lookup_hot([['Paris'],['Paris'],['Auckland'],['Montreal']])
print('One Hot Encoding result:')
print(str_lookup_res)

str_lookup_layer=tf.keras.layers.StringLookup(num_oov_indices=5)#this doesn't assign all unknown categories as 0
#can handle upto 5 unexpected ccategories
str_lookup_layer.adapt(cities)
print('Result of string lookup layer after adding num_oov_indices:')
print(str_lookup_layer([["Paris"],["Auckland"],["Foo"],["Bar"],["Baz"]]))
#since num_oov_indices=5, Paris gets classified as 5
#the unknown ones are classified to category numbers<5

#HASHING LAYER
hashing_layer=tf.keras.layers.Hashing(num_bins=10)
hashing_res=hashing_layer([["Paris"],["Auckland"],['Auckland'],['Poland']])
print('Results of hashing layer on city names:')
print(hashing_res)

Tensors age:
tf.Tensor(
[[10.]
 [93.]
 [57.]
 [18.]
 [37.]
 [ 5.]], shape=(6, 1), dtype=float32)
Age after discretization:
tf.Tensor(
[[0]
 [2]
 [2]
 [1]
 [1]
 [0]], shape=(6, 1), dtype=int64)
Individual tensors in age_categories:
tf.Tensor([0], shape=(1,), dtype=int64)
tf.Tensor([2], shape=(1,), dtype=int64)
tf.Tensor([2], shape=(1,), dtype=int64)
tf.Tensor([1], shape=(1,), dtype=int64)
tf.Tensor([1], shape=(1,), dtype=int64)
tf.Tensor([0], shape=(1,), dtype=int64)
Age categories after specifying only number of bins:
tf.Tensor(
[[1]
 [2]
 [2]
 [1]
 [2]
 [0]], shape=(6, 1), dtype=int64)
After doing CategoryEncoding on age_categories
tf.Tensor(
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]], shape=(6, 3), dtype=float32)
Conversion of two dimensional age categories to CategoryEncoding:
tf.Tensor(
[[1. 1. 0.]
 [0. 0. 2.]
 [1. 0. 1.]], shape=(3, 3), dtype=float32)
Result of stringlookup on new cities data:
tf.Tensor(
[[1]
 [1]
 [3]
 [0]], shape=(4, 1), dtype=int64

In [23]:
#EMBEDDING
tf.random.set_seed(42)
embedding_layer=tf.keras.layers.Embedding(input_dim=5,output_dim=2)
#input_dim specifies max no of catgroy IDs, lesser number can also be used
print('Results of embedding:')
print(embedding_layer(np.array([2,4,2])))
#here 2,4,2 are category IDs
#there is one row per category and one column per dimension in sparse matrix
ocean_prox = [['NEAR BAY'], ['NEAR OCEAN'], ['INLAND'], ['ISLAND']]
str_lookup = tf.keras.layers.StringLookup(num_oov_indices=5)
str_lookup.adapt(ocean_prox)

# Encode string categories into integers
encoded_inputs=str_lookup(np.array([['NEAR OCEAN'],['ISLAND'],['NEAR BAY']]))
encoded_inputs=tf.clip_by_value(encoded_inputs,0,4)  # Clip indices to [0, input_dim)

lookup_embed=tf.keras.Sequential([tf.keras.layers.Embedding(input_dim=str_lookup.vocabulary_size()
                                                            ,output_dim=2)])
results=lookup_embed(encoded_inputs)
print('Result of combining lookup and embedding keras layers:')
print(results)

#TEXT PREPROCESSING
sentence_data=["To be","!(to be)","Be,be,be","That's the question"]
text_vec_layer=tf.keras.layers.TextVectorization()
text_vec_layer.adapt(sentence_data)

text_results=text_vec_layer(['Be good!','To the question'])
print('Results of text vectorization:')
print(text_results)
#unknown words get encoded as 1, 0 is padded as first input is shorter than second
#tf idf gives importance(weight of aparticular word)

text_vec_Layer=tf.keras.layers.TextVectorization(output_mode='tf_idf')
text_vec_layer.adapt(sentence_data)
print('TF_IDF RESULTS:')
print(text_vec_layer(["Be good!","Question: be or be?"]))
#the more a word appears, the lesser the tf_idf score

#USING PRE TRAINED MODEL FROM TENSOR HUB
import tensorflow_hub as hub
hub_layer=hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2")
#this layer outputs roughly 50 dimensions
sentence_embeddings=hub_layer(tf.constant(["To be","Not to be"]))
print('Results from the pre trained model:')
print(sentence_embeddings.numpy.round(2))

Results of embedding:
tf.Tensor(
[[-0.04512222  0.04537706]
 [-0.04893095  0.04836159]
 [-0.04512222  0.04537706]], shape=(3, 2), dtype=float32)
Result of combining lookup and embedding keras layers:
tf.Tensor(
[[[ 0.00085356 -0.01974626]]

 [[ 0.00085356 -0.01974626]]

 [[ 0.00085356 -0.01974626]]], shape=(3, 1, 2), dtype=float32)
Results of text vectorization:
tf.Tensor(
[[3 1 0]
 [2 4 6]], shape=(2, 3), dtype=int64)
TF_IDF RESULTS:
tf.Tensor(
[[3 1 0 0]
 [6 3 1 3]], shape=(2, 4), dtype=int64)
Results from the pre trained model:


AttributeError: 'function' object has no attribute 'round'

In [24]:
#IMAGE PREPROCESSING LAYERS
from sklearn.datasets import load_sample_images
images=load_sample_images()['images']#multi dim array
crop_image_layer=tf.keras.layers.CenterCrop(height=100,width=100)
cropped_images=crop_image_layer(images)

import tensorflow_datasets as tfds
datasets=tfds.load(name='mnist')
mnist_train,mnist_test=datasets['train'],datasets['test']

for batch in mnist_train.shuffle(10_000,seed=42).batch(32).prefetch(1):
    images=batch['image']
    label=batch['label']
#each item here is a dictionary containing features and labels
#for keras it should ideally be a tuple containing 2 elements
mnist_train=mnist_train.shuffle(10_000,seed=42).batch(32)
mnist_train=mnist_train.map(lambda items: (items['image'],items['label']))
mnist_train=mnist_train.prefetch(1)

#you can directly do this while loading if you use as_supervised=True
train_set,valid_set,test_set=tfds.load(name='mnist',
                                      split=["train[:90%]","train[90%:]","test"],
                                      as_supervised=True)
train_set=train_set.shuffle(10_000,seed=42).batch(32).prefetch(1)
valid_set=valid_set.batch(32).cache()
test_set=test_set.batch(32).cache()

tf.random.set_seed(42)
model=tf.keras.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                          tf.keras.layers.Dense(10,activation='softmax')])
model.compile(loss='sparse_categorical_crossentropy',optimizer='nadam',metrics=['accuracy'])
history=model.fit(train_set,validation_data=valid_set,epochs=5)
print('Test set evaluation:')
print(model.evaluate(test_set))

[array([[[174, 201, 231],
         [174, 201, 231],
         [174, 201, 231],
         ...,
         [250, 251, 255],
         [250, 251, 255],
         [250, 251, 255]],
 
        [[172, 199, 229],
         [173, 200, 230],
         [173, 200, 230],
         ...,
         [251, 252, 255],
         [251, 252, 255],
         [251, 252, 255]],
 
        [[174, 201, 231],
         [174, 201, 231],
         [174, 201, 231],
         ...,
         [252, 253, 255],
         [252, 253, 255],
         [252, 253, 255]],
 
        ...,
 
        [[ 88,  80,   7],
         [147, 138,  69],
         [122, 116,  38],
         ...,
         [ 39,  42,  33],
         [  8,  14,   2],
         [  6,  12,   0]],
 
        [[122, 112,  41],
         [129, 120,  53],
         [118, 112,  36],
         ...,
         [  9,  12,   3],
         [  9,  15,   3],
         [ 16,  24,   9]],
 
        [[116, 103,  35],
         [104,  93,  31],
         [108, 102,  28],
         ...,
         [ 43,  49,  39],
  